# Hidden Markov Model for Named Entity Recognition
This notebook implements a Hidden Markov Model (HMM) for Named Entity Recognition (NER). It covers training, prediction using the Viterbi algorithm, and evaluation of model accuracy.

## Imports

In [ ]:
import re
import ast
import math
from collections import defaultdict
import pandas as pd

## Hidden Markov Model Class
This class encapsulates the entire functionality of the HMM.

In [ ]:

class HiddenMarkovModel:
    def __init__(self, smoothing=1e-6):
        """Initialize HMM with required components."""
        self.tag_counts = defaultdict(int)
        self.word_tag_map = {}
        self.states = []
        self.trans_probs = defaultdict(lambda: defaultdict(lambda: smoothing))
        self.emit_probs = defaultdict(lambda: defaultdict(lambda: smoothing))
        self.start_probs = defaultdict(lambda: smoothing)
        self.smoothing = smoothing

    def train(self, sentences, tagged_sentences):
        """Train HMM by counting frequencies and computing probabilities."""
        self._count_tags_and_words(sentences, tagged_sentences)
        self._compute_transition_probs(tagged_sentences)
        self._compute_emission_probs(sentences, tagged_sentences)
        self._compute_start_probs(tagged_sentences)
        self.states = list(self.tag_counts.keys())

    def viterbi(self, sentence):
        """Predict tag sequence using the Viterbi algorithm."""
        if not sentence:
            return []
        V = [{}]
        path = {}
        first_word = sentence[0]
        for state in self.states:
            V[0][state] = math.log(self.start_probs[state]) + math.log(self.emit_probs[state][first_word])
            path[state] = [state]
        for t in range(1, len(sentence)):
            V.append({})
            new_path = {}
            for curr_state in self.states:
                max_prob = -math.inf
                best_prev = None
                for prev_state in self.states:
                    prob = V[t-1][prev_state] + math.log(self.trans_probs[prev_state][curr_state]) + math.log(self.emit_probs[curr_state][sentence[t]])
                    if prob > max_prob:
                        max_prob = prob
                        best_prev = prev_state
                V[t][curr_state] = max_prob
                new_path[curr_state] = path[best_prev] + [curr_state]
            path = new_path
        last_col = V[-1]
        best_final = max(last_col.keys(), key=lambda s: last_col[s])
        return path[best_final]

    def _count_tags_and_words(self, sentences, tagged_sentences):
        """Count tag frequencies and build word-tag map."""
        word_tag_counts = defaultdict(lambda: defaultdict(int))
        for words, tags in zip(sentences, tagged_sentences):
            for word, tag in zip(words, tags):
                self.tag_counts[tag] += 1
                word_tag_counts[word][tag] += 1
        for word in word_tag_counts:
            self.word_tag_map[word] = max(word_tag_counts[word].items(), key=lambda x: x[1])[0]

    def _compute_transition_probs(self, tagged_sentences):
        """Compute transition probabilities: P(tag_t | tag_t-1)."""
        transitions = defaultdict(lambda: defaultdict(int))
        for tags in tagged_sentences:
            for i in range(len(tags)-1):
                curr, next_tag = tags[i], tags[i+1]
                transitions[curr][next_tag] += 1
        for curr in transitions:
            total = sum(transitions[curr].values())
            for next_tag in transitions[curr]:
                self.trans_probs[curr][next_tag] = transitions[curr][next_tag] / total

    def _compute_emission_probs(self, sentences, tagged_sentences):
        """Compute emission probabilities: P(word | tag)."""
        emissions = defaultdict(lambda: defaultdict(int))
        for words, tags in zip(sentences, tagged_sentences):
            for word, tag in zip(words, tags):
                emissions[tag][word] += 1
        for tag in emissions:
            total = self.tag_counts[tag]
            for word in emissions[tag]:
                self.emit_probs[tag][word] = emissions[tag][word] / total

    def _compute_start_probs(self, tagged_sentences):
        """Compute start tag probabilities."""
        start_counts = defaultdict(int)
        total = len(tagged_sentences)
        for tags in tagged_sentences:
            if tags:
                start_counts[tags[0]] += 1
        for tag in start_counts:
            self.start_probs[tag] = start_counts[tag] / total


## Data Loading and Preprocessing
This function loads and parses a CSV-formatted NER dataset.

In [ ]:

def load_data(filepath):
    """Load and preprocess NER dataset."""
    df = pd.read_csv(filepath)
    sentences = df['Sentence'].apply(str.split).tolist()
    tagged_sentences = df['Tag'].apply(ast.literal_eval).tolist()
    return sentences, tagged_sentences


## Evaluation
This function computes the model's tagging accuracy.

In [ ]:

def evaluate(model, test_sentences, test_tags):
    """Calculate model accuracy."""
    correct = 0
    total = 0
    for words, true_tags in zip(test_sentences, test_tags):
        pred_tags = model.viterbi(words)
        correct += sum(1 for t1, t2 in zip(true_tags, pred_tags) if t1 == t2)
        total += len(true_tags)
    return correct / total


## Main Execution
Train the HMM, evaluate its performance, and test a sample sentence.

In [ ]:

def main():
    sentences, tagged_sentences = load_data("ner.csv")
    split_idx = int(0.8 * len(sentences))
    train_sent = sentences[:split_idx]
    train_tags = tagged_sentences[:split_idx]
    test_sent = sentences[split_idx:]
    test_tags = tagged_sentences[split_idx:]

    hmm = HiddenMarkovModel()
    hmm.train(train_sent, train_tags)

    accuracy = evaluate(hmm, test_sent, test_tags)
    print(f"Model Accuracy: {accuracy:.2%}")

    test_sentence = ["Manila", "is", "the", "capital", "of", "the", "Philippines"]
    print("\nTest Sentence:", test_sentence)
    print("Predicted Tags:", hmm.viterbi(test_sentence))

main()
